<a href="https://colab.research.google.com/github/ankit-rathi/AR-Talks/blob/master/Postgres_CRUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,742 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [50.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,343 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [2]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "DROP ROLE root;"
!sudo -u postgres psql -U postgres -c "CREATE ROLE root WITH SUPERUSER;"
!sudo -u postgres psql -U postgres -c "ALTER ROLE root WITH LOGIN;"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

ERROR:  role "root" does not exist
CREATE ROLE
ALTER ROLE
ALTER ROLE


In [3]:
!sudo -u postgres psql -c "DROP TABLE api_req_res;"

ERROR:  table "api_req_res" does not exist


In [4]:
!sudo -u postgres psql -c "CREATE TABLE api_req_res ( req VARCHAR(255), id INTEGER, req_json VARCHAR(255), res_json VARCHAR(255) );"

CREATE TABLE


In [5]:
#!/usr/bin/env python3
import sys, psycopg2, psycopg2.sql as sql


class Crud:
    def __init__(self, user, password, host, port, dbname, table, primarykey):
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.dbname = dbname
        self.table = table
        self.primarykey = primarykey



    def connect(self):
        try:
            connection = psycopg2.connect(
                user = self.user,
                password = self.password,
                host = self.host,
                port = self.port,
                dbname = self.dbname
            )
            cursor = connection.cursor()
            print(
                '------------------------------------------------------------'
                '\n-# PostgreSQL connection & transaction is ACTIVE\n'
                )
        except (Exception, psycopg2.Error) as error :
            print(error, error.pgcode, error.pgerror, sep = '\n')
            sys.exit()
        else:
            self._connection = connection
            self._cursor = cursor
            self._counter = 0


    def _check_connection(self):
        try:
            self._connection
        except AttributeError:
            print('ERROR: NOT Connected to Database')
            sys.exit()


    def _execute(self, query, Placeholder_value = None):
        self._check_connection()
        if Placeholder_value == None or None in Placeholder_value:
            self._cursor.execute(query)
            print( '-# ' + query.as_string(self._connection) + ';\n' )
        else:
            self._cursor.execute(query, Placeholder_value)
            print( '-# ' + query.as_string(self._connection) % Placeholder_value + ';\n' )


    def commit(self):
        self._check_connection()
        self._connection.commit()
        print('-# COMMIT '+ str(self._counter) +' changes\n')
        self._counter = 0


    def close(self, commit = False):
        self._check_connection()
        if commit:
            self.commit()
        else:
            self._cursor.close()
            self._connection.close()
        if self._counter > 0:
            print(
                '-# '+ str(self._counter) +' changes NOT commited  CLOSE connection\n'
                '------------------------------------------------------------\n'
            )
        else:
            print(
                '-# CLOSE connection\n'
                '------------------------------------------------------------\n'
            )


    def insert(self, **column_value):
        insert_query  = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(self.table),
            sql.SQL(', ').join( map( sql.Identifier, column_value.keys() ) ),
            sql.SQL(', ').join(sql.Placeholder() * len(column_value.values()))
        )
        record_to_insert = tuple(column_value.values())
        self._execute(insert_query, record_to_insert )
        self._counter += 1


    def insert_many(self, columns, rows):
        insert_query  = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(self.table),
            sql.SQL(', ').join( map( sql.Identifier, columns ) ),
            sql.SQL(', ').join(sql.Placeholder() * len(rows[0]))
        )
        for row in rows:
            row = tuple(row)
            self._execute(insert_query, row )
            self._counter += 1


    def select(self, columns, primaryKey_value = None):
        if primaryKey_value == None:
            select_query = sql.SQL("SELECT {} FROM {}").format(
                sql.SQL(',').join(map(sql.Identifier, columns)),
                sql.Identifier(self.table)
            )
            self._execute( select_query )
        else:
            select_query = sql.SQL("SELECT {} FROM {} WHERE {} = {}").format(
                sql.SQL(',').join(map(sql.Identifier, columns)),
                sql.Identifier(self.table),
                sql.Identifier(self.primarykey),
                sql.Placeholder()
            )
            self._execute( select_query, ( primaryKey_value,))
        try:
            selected = self._cursor.fetchall()
        except psycopg2.ProgrammingError as error:
            selected = '# ERROR: ' + str(error)
        else:
            print('-# ' + str(selected) + '\n')
            return selected


    def select_all(self, primaryKey_value = None):
        if primaryKey_value == None:
            select_query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(self.table))
            self._execute( select_query )
        else:
            select_query = sql.SQL("SELECT * FROM {} WHERE {} = {}").format(
                sql.Identifier(self.table),
                sql.Identifier(self.primarykey),
                sql.Placeholder()
            )
            self._execute( select_query, ( primaryKey_value,))
        try:
            selected = self._cursor.fetchall()
        except psycopg2.ProgrammingError as error:
            selected = '# ERROR: ' + str(error)
        else:
            print('-# ' + str(selected) + '\n')
            return selected


    def update(self, column, column_value, primaryKey_value):
        update_query  = sql.SQL("UPDATE {} SET {} = {} WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.Identifier(column),
            sql.Placeholder(),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        self._execute(update_query, ( column_value, primaryKey_value))
        self._counter += 1


    def update_multiple_columns(self, columns, columns_value, primaryKey_value):
        update_query  = sql.SQL("UPDATE {} SET ({}) = ({}) WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.SQL(',').join(map(sql.Identifier, columns)),
            sql.SQL(', ').join(sql.Placeholder() * len(columns_value)),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        Placeholder_value = list(columns_value)
        Placeholder_value.append(primaryKey_value)
        Placeholder_value = tuple(Placeholder_value)
        self._execute(update_query, Placeholder_value)
        self._counter += 1


    def delete(self, primaryKey_value):
        delete_query  = sql.SQL("DELETE FROM {} WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        self._execute(delete_query, ( primaryKey_value,))
        self._counter += 1


    def delete_all(self):
        delete_query  = sql.SQL("DELETE FROM {}").format( sql.Identifier(self.table) )
        self._execute(delete_query)
        self._counter += 1

In [6]:
table = Crud(
    user = 'postgres',
    password = 'postgres',
    host = '127.0.0.1',
    port = '5432',
    dbname = 'postgres',
    table = 'api_req_res',
    primarykey = 'req'
)


table.connect()


table.insert(
    req = 'Req1234',
    id = 1,
    req_json = 'request_json1',
    res_json = 'response_json1'
)


table.insert_many(
    columns = ('req', 'id', 'req_json', 'res_json'),
    rows = [
        ['Req5678', 1, 'request_json2', 'response_json2'],
        ['Req7891', 1, 'request_json3', 'response_json3'],
    ]
)


table.commit()

'''
table.select_all()


table.select_all(
    primaryKey_value = 'Req5678'
)


table.select(
    columns = ['req_json'],
    primaryKey_value = 'Req1234'
)


table.select(
    columns = ['req']
)


table.update(
    column = 'res_json',
    column_value = 'response_json6',
    primaryKey_value = 'Req5678'
)


table.update_multiple_columns(
    columns = ['req_json', 'res_json'],
    columns_value = ['request_json9', 'response_json9'],
    primaryKey_value = 'Req1234'
)


table.delete(
    primaryKey_value = 'Req7891'
)


table.select_all()


table.delete_all()


table.close("commit")
'''

------------------------------------------------------------
-# PostgreSQL connection & transaction is ACTIVE

-# INSERT INTO "api_req_res" ("req", "id", "req_json", "res_json") VALUES (Req1234, 1, request_json1, response_json1);

-# INSERT INTO "api_req_res" ("req", "id", "req_json", "res_json") VALUES (Req5678, 1, request_json2, response_json2);

-# INSERT INTO "api_req_res" ("req", "id", "req_json", "res_json") VALUES (Req7891, 1, request_json3, response_json3);

-# COMMIT 3 changes



'\ntable.select_all()\n\n\ntable.select_all(\n    primaryKey_value = \'Req5678\'\n)\n\n\ntable.select(\n    columns = [\'req_json\'],\n    primaryKey_value = \'Req1234\'\n)\n\n\ntable.select(\n    columns = [\'req\']\n)\n\n\ntable.update(\n    column = \'res_json\',\n    column_value = \'response_json6\',\n    primaryKey_value = \'Req5678\'\n)\n\n\ntable.update_multiple_columns(\n    columns = [\'req_json\', \'res_json\'],\n    columns_value = [\'request_json9\', \'response_json9\'],\n    primaryKey_value = \'Req1234\'\n)\n\n\ntable.delete(\n    primaryKey_value = \'Req7891\'\n)\n\n\ntable.select_all()\n\n\ntable.delete_all()\n\n\ntable.close("commit")\n'

In [7]:
table.commit()
table.select_all()

-# COMMIT 0 changes

-# SELECT * FROM "api_req_res";

-# [('Req1234', 1, 'request_json1', 'response_json1'), ('Req5678', 1, 'request_json2', 'response_json2'), ('Req7891', 1, 'request_json3', 'response_json3')]



[('Req1234', 1, 'request_json1', 'response_json1'),
 ('Req5678', 1, 'request_json2', 'response_json2'),
 ('Req7891', 1, 'request_json3', 'response_json3')]

In [8]:
#!sudo -u postgres psql -c "DROP TABLE api_req_res;"

In [9]:
'''
Class Crud:
    Method __init__(user, password, host, port, dbname, table, primarykey):
        Initialize attributes: user, password, host, port, dbname, table, primarykey

    Method connect():
        Try connecting to PostgreSQL database using provided credentials
        If successful, print connection status message
        If connection fails, print error message and exit

    Method _check_connection():
        Check if connection to database has been established
        If not, print error message and exit

    Method _execute(query, Placeholder_value = None):
        Check connection to database
        If Placeholder_value is None or contains None, execute query without placeholders
        Else, execute query with placeholders and Placeholder_value
        Print executed query

    Method commit():
        Check connection to database
        Commit changes and print commit message
        Reset counter to 0

    Method close(commit = False):
        Check connection to database
        If commit is True, call commit() method
        If commit is False, close cursor and connection
        If changes were not committed, print message indicating changes were not committed

    Method insert(**column_value):
        Construct INSERT query with table name and column-value pairs
        Execute the query to insert data into the table
        Increment counter for changes made

    Method insert_many(columns, rows):
        Construct INSERT query with table name, columns, and rows
        Execute the query to insert multiple rows into the table
        Increment counter for changes made

    Method select(columns, primaryKey_value = None):
        Construct SELECT query with table name, columns, and optional primary key value
        Execute the query to fetch data from the table
        Return the fetched rows

    Method select_all(primaryKey_value = None):
        Construct SELECT query to fetch all rows from the table
        Execute the query and return the fetched rows

    Method update(column, column_value, primaryKey_value):
        Construct UPDATE query with table name, column, column value, and primary key value
        Execute the query to update data in the table
        Increment counter for changes made

    Method update_multiple_columns(columns, columns_value, primaryKey_value):
        Construct UPDATE query with table name, columns, column values, and primary key value
        Execute the query to update data in the table for multiple columns
        Increment counter for changes made

    Method delete(primaryKey_value):
        Construct DELETE query with table name and primary key value
        Execute the query to delete data from the table
        Increment counter for changes made

    Method delete_all():
        Construct DELETE query to delete all data from the table
        Execute the query to delete all rows
        Increment counter for changes made

# Example usage:
If __name__ == "__main__":
    Create an instance of Crud with database credentials
    Connect to the database
    Perform CRUD operations as needed
    Close the connection and commit changes if necessary

'''

'\nClass Crud:\n    Method __init__(user, password, host, port, dbname, table, primarykey):\n        Initialize attributes: user, password, host, port, dbname, table, primarykey\n\n    Method connect():\n        Try connecting to PostgreSQL database using provided credentials\n        If successful, print connection status message\n        If connection fails, print error message and exit\n\n    Method _check_connection():\n        Check if connection to database has been established\n        If not, print error message and exit\n\n    Method _execute(query, Placeholder_value = None):\n        Check connection to database\n        If Placeholder_value is None or contains None, execute query without placeholders\n        Else, execute query with placeholders and Placeholder_value\n        Print executed query\n\n    Method commit():\n        Check connection to database\n        Commit changes and print commit message\n        Reset counter to 0\n\n    Method close(commit = False):\n   